In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import beta
from scipy.stats import chi2
from scipy.stats import gamma
from scipy.stats import triang

In [2]:
types = ['ball', 'bike', 'blocks', 'book', 'coal', 'doll', 'gloves', 'horse', 'train']
num_types = len(types)
key_to_item = dict((key, index) for key, index in zip(types, range(len(types))))
step = 0.1

In [3]:
def normal(x, mu, sigma):
    if x > 0:
        return norm.pdf(x, mu, sigma)
    return norm.cdf(0, mu, sigma)

weight_gen_map = {
    'horse': lambda: max(0, np.random.normal(5,2,1)[0]),
    'ball': lambda: max(0, 1 + np.random.normal(1,0.3,1)[0]),
    'bike': lambda: max(0, np.random.normal(20,10,1)[0]),
    'train': lambda: max(0, np.random.normal(10,5,1)[0]),
    'coal': lambda: 47 * np.random.beta(0.5,0.5,1)[0],
    'book': lambda: np.random.chisquare(2,1)[0],
    'doll': lambda: np.random.gamma(5,1,1)[0],
    'blocks': lambda: np.random.triangular(5,10,20,1)[0],
    'gloves': lambda: 3.0 + np.random.rand(1)[0] if np.random.rand(1) < 0.3 else np.random.rand(1)[0]
}

distribution_map = {
    'horse': lambda x: normal(x, *(5, 2)),
    'ball': lambda x: normal(x - 1, *(2, 0.3)),
    'bike': lambda x: normal(x, *(20,10)),
    'train': lambda x: normal(x, *(10,5)),
    'coal': lambda x: beta.pdf(x / 47 , 0.5, 0.5),
    'book': lambda x: chi2.pdf(x, 2),
    'doll': lambda x: (x**4) * np.exp(-x) / 25,
    'blocks': lambda x: triang.pdf(x, c = 1.0/3, loc = 5, scale = 15),
    'gloves': lambda x: 0.7 if (0 < x and x < 1) else 0.3 if (3 < x and x < 4) else 0
}

In [4]:
def save_expected_weights():
    expected_weight_list = [list(key) + [expected_weight_map[key]] for key in expected_weight_map]
    pd.DataFrame(expected_weight_list).to_csv('data/expected_weights', index=False)

def load_expected_weights():
    expected_weight_list = pd.read_csv('data/expected_weights')
    return dict([(tuple(int(val) for val in row[:9]), row[9]) for row in expected_weight_list.values])

In [5]:
def bag_to_key(bag):
    return tuple(bag)

def item_to_key(item):
    return tuple(1 if item == i else 0 for i in range(num_types))

def item_to_full_item(item):
    item_full = np.zeros((num_types,), dtype=np.int)
    item_full[item] += 1
    return item_full

In [6]:
def get_expected(bag):
    key = bag_to_key(bag)
    if key in expected_weight_map:
        return expected_weight_map[key]
    expected_weight = sum(prob * weight for prob, weight in zip(weight_map[key], weight_slices))
    expected_weight_map[key] = expected_weight
    return expected_weight

def get_chance_too_full(bag):
    key = bag_to_key(bag)
    if key in chance_too_full_map:
        return chance_too_full_map[key]
    chance_too_full = sum(weight_map[key])
    chance_too_full_map[key] = chance_too_full
    return 1 - chance_too_full

def calculate_combined_distribution(old_bag, item):
    bag = old_bag.copy()
    bag[item] += 1
    key = bag_to_key(bag)
    # already caculated
    if key in weight_map:
        return None
    
    distribution = np.zeros((500,), dtype=np.float)
    d1 = weight_map[bag_to_key(old_bag)]
    d2 = weight_array_singles[item]
    for p1, w1 in zip(d1, weight_indices):
        for p2, w2 in zip(d2, weight_indices):
            new_weight = w1 + w2
            if new_weight < 500:
                distribution[new_weight] += (p1 * p2)
    weight_map[key] = distribution
    return bag

# TODOS
- make fill_bags faster by 
    - add method to give chance of going over
    - for all values with a certain chance of going over add it plus given item to expected_map
- try increasing variance by increasing expected value if high chance of going over (multiply expected by (1/chance_under)^(0.5))
- iteratively try different subset swaps
- actually submit

In [7]:
# use this to populate map with combinations
# TODO make this faster and lower trial area
def hydrate_map(iterations=10):
    last_bags = [np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])]
    new_bags = []
    for i in range(iterations):
        for bag in last_bags:
            expected = get_expected(bag)
            for j in range(num_types):
                new_bag = calculate_combined_distribution(bag, j)
                if new_bag != None and get_expected(new_bag) > expected:
                    new_bags.append(new_bag)
        last_bags = new_bags
        new_bags = []

In [8]:
weight_map = {tuple([0] * num_types): np.zeros((500,), dtype=np.float)}
weight_map[tuple([0] * num_types)][0] = 1
expected_weight_map = {tuple([0] * num_types): 0}
chance_too_full_map = {tuple([0] * num_types): 0}

weight_slices = np.arange(step / 2, 50, step)
weight_indices = list(range(500))
weight_array_singles = [np.array([step * distribution_map[key](weight) for weight in weight_slices]) for key in types]

In [ ]:
# Normalize to make prop 1 by dividing by get_chance_too_full

weight_slices = np.arange(step / 2, 50, step)
weight_indices = list(range(500))
weight_array_singles = [np.array([step * distribution_map[key](weight) for weight in weight_slices]) for key in types]

In [53]:
%%time
hydrate_map(3)
print (len(weight_map))

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


220
CPU times: user 17.7 s, sys: 44.3 ms, total: 17.7 s
Wall time: 18.1 s


In [55]:
%%time
hydrate_map(5)
print (len(weight_map))

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


1891
CPU times: user 2min 31s, sys: 883 ms, total: 2min 32s
Wall time: 2min 36s


In [ ]:
%%time
hydrate_map(50)
print (len(weight_map))

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [19]:
def normalize(key, usefulness):
    return expected_weight_map[key] - sum(float(item) * factor for item, factor in zip(key, usefulness))

def accumulate(rows):
    counts = np.array([0] * num_types)
    score = 0
    for row in rows:
        counts += np.array(row[0])
        score += row[1]
    print (len(rows))
    print (counts)
    print (np.array([1100, 500, 1000, 1200, 166, 1000, 200, 1000, 1000]) - counts)
    print (score)

In [20]:
# add normally as same in test_usefullness
# iteratively
    # remove some subset of elements and add in other possible combinations

In [182]:
def test_usefullness(usefulness):
    sorted_normalized = sorted(expected_weight_map, key=lambda key: normalize(key, usefulness) )
    sorted_normalized.reverse()

    items_left = np.array([1100, 500, 1000, 1200, 166, 1000, 200, 1000, 1000])
    score = 0
    items = 0
    bag_counts = []
    for row in sorted_normalized:
        num_of_item = 0
        while items < 1000 and sum(item < 0 for item in (items_left - np.array(row))) == 0:
            items_left -= np.array(row)
            score += expected_weight_map[row]
            items += 1
            num_of_item += 1
        
        if num_of_item > 0:
            #bag_counts.append((num_of_item, (row, expected_weight_map[row])))
            bag_counts.append((num_of_item, row))
        
        if items == 1000:
            break

    return (score, bag_counts)

In [36]:
usefulness_base = [0.6, -1.7, 1.4, 1.45, -0.7, -0.15, -0.7, 0.55, -0.5]
jitters = [0, -0.05, 0.05, 0.1, -0.1]

for i in range(4):
    best_usefulness = None
    best_score = 0
    for j in range(len(usefulness_base)):
        for k in range(j, len(usefulness_base)):
            for jitter1 in jitters:
                for jitter2 in jitters:
                    usefulness = usefulness_base.copy()
                    usefulness[j] += jitter1
                    usefulness[k] += jitter2
                    score, bag_counts = test_usefullness(usefulness)
                    if score > best_score:
                        best_score = score
                        best_usefulness = usefulness

    print (best_usefulness)
    print (best_score)
    usefulness_base = best_usefulness

[0.6, -1.7, 1.4000000000000001, 1.45, -0.7, -0.15, -0.7, 0.55, -0.5]
32208.5902146


KeyboardInterrupt: 

In [138]:
def score_bag(bag_counts):
    assert (sum(count for count, bag in bag_counts) == 1000)
    score_sum = 0
    for count, bag in bag_counts:
        for i in range(count):
            weight = sum(weight_gen_map[item_type]() for item_type, item_num in zip(types, bag) for j in range(item_num))
            if weight > 50:
                continue
            score_sum += weight
    return score_sum

In [139]:
score, bag_counts = test_usefullness([0.5, -1.7, 1.3, 1.45, -0.7, -0.15, -0.7, 0.55, -0.5])
print (max(score_bag(bag_counts) for i in range(10)))
print (score)

31786.4814366
32191.9035177


In [140]:
bag_counts

[(200, (3, 0, 2, 6, 0, 0, 0, 0, 0)),
 (21, (23, 0, 0, 0, 0, 0, 0, 0, 0)),
 (8, (2, 0, 3, 0, 0, 0, 0, 0, 0)),
 (192, (0, 0, 3, 0, 0, 1, 0, 0, 0)),
 (269, (0, 1, 0, 0, 0, 3, 0, 0, 0)),
 (115, (0, 2, 0, 0, 0, 0, 0, 0, 0)),
 (1, (1, 1, 0, 0, 0, 1, 0, 0, 0)),
 (166, (0, 0, 0, 0, 1, 0, 0, 0, 0)),
 (28, (0, 0, 0, 0, 0, 0, 0, 0, 1))]

In [185]:
usefulness = [0.6, -1.7, 1.4, 1.45, -0.7, -0.15, -0.7, 0.55, -0.5]
sorted_normalized = sorted(expected_weight_map, key=lambda key: normalize(key, usefulness) )
sorted_normalized.reverse()

In [188]:
sorted_normalized = sorted(expected_weight_map, key=lambda key: expected_weight_map[key])
sorted_normalized.reverse()

In [189]:
[(a, expected_weight_map[a]) for a in sorted_normalized][:20]

[((11, 0, 0, 0, 1, 1, 0, 0, 0), nan),
 ((2, 0, 0, 10, 0, 0, 0, 0, 1), 53.338088232424653),
 ((0, 0, 0, 11, 0, 2, 0, 1, 0), 53.20608199314939),
 ((5, 0, 0, 1, 0, 0, 0, 1, 2), 34.58874623790323),
 ((7, 0, 1, 1, 0, 0, 2, 2, 0), 21.123966234117738),
 ((3, 0, 0, 6, 1, 0, 0, 1, 1), nan),
 ((1, 0, 0, 9, 0, 2, 1, 2, 0), 39.354008523834658),
 ((0, 0, 0, 14, 1, 1, 5, 0, 0), nan),
 ((1, 0, 1, 1, 1, 0, 4, 3, 0), nan),
 ((3, 0, 1, 0, 1, 0, 2, 1, 1), nan),
 ((0, 1, 0, 0, 1, 3, 0, 0, 0), nan),
 ((1, 0, 0, 3, 1, 5, 0, 0, 0), nan),
 ((3, 0, 0, 14, 1, 0, 4, 0, 0), nan),
 ((7, 0, 0, 11, 0, 0, 0, 1, 0), 52.064875814323784),
 ((3, 0, 0, 8, 0, 1, 0, 0, 1), 48.379097720237169),
 ((1, 0, 2, 1, 0, 2, 0, 0, 0), 35.674275303204212),
 ((0, 0, 1, 0, 0, 1, 0, 3, 0), 29.944803409912744),
 ((3, 0, 0, 4, 0, 1, 2, 3, 0), 27.166941293789307),
 ((1, 0, 0, 1, 0, 3, 1, 4, 0), 27.013106547395317),
 ((2, 0, 1, 3, 0, 3, 0, 0, 1), 25.863402203273761)]

In [175]:
get_expected((2, 0, 1, 7, 0, 0, 0, 0, 1))

47.48061178056939

In [173]:
get_combined_distribution([0, 0, 0, 1, 0, 0, 0, 0, 0], 3)
get_combined_distribution([0, 0, 0, 2, 0, 0, 0, 0, 0], 3)
get_combined_distribution([0, 0, 0, 3, 0, 0, 0, 0, 0], 3)
get_combined_distribution([0, 0, 0, 4, 0, 0, 0, 0, 0], 0)
get_combined_distribution([1, 0, 0, 4, 0, 0, 0, 0, 0], 0)
get_combined_distribution([2, 0, 0, 4, 0, 0, 0, 0, 0], 3)
get_combined_distribution([2, 0, 0, 5, 0, 0, 0, 0, 0], 3)
get_combined_distribution([2, 0, 0, 6, 0, 0, 0, 0, 0], 3)
get_combined_distribution([2, 0, 0, 7, 0, 0, 0, 0, 0], 8)
get_combined_distribution([2, 0, 0, 7, 0, 0, 0, 0, 1], 3)
get_combined_distribution([2, 0, 0, 7, 0, 0, 0, 0, 1], 2)
get_combined_distribution([2, 0, 1, 7, 0, 0, 0, 0, 1], 2)
get_combined_distribution([2, 0, 1, 7, 0, 0, 0, 0, 1], 3)

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   3.32337005e-37,
         3.90577832e-36,   2.48449688e-35,   1.13386006e-34,
         4.15552727e-34,   1.29883834e-33,   4.62069359e-33,
         6.50493562e-32,   1.83019076e-30,   3.93730541e-29,
         5.97292817e-28,   6.54549203e-27,   5.36588954e-26,
         3.41604448e-25,   1.75423463e-24,   7.62020873e-24,
         3.17537428e-23,   1.88566648e-22,   2.08828122e-21,
         2.79386546e-20,   3.35804555e-19,   3.40717011e-18,
         2.90446140e-17,